In [1]:
import os
import requests

In [2]:
# Only once -- download the tiny shakespeare dataset:

# input_file_path = os.path.join(os.getcwd(), 'input.txt')
# if not os.path.exists(input_file_path):
#     data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
#     with open(input_file_path, 'w', encoding='utf-8') as f:
#         f.write(requests.get(data_url).text)

In [3]:
# Read and inspect the data:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [4]:
print('Length of the dataset in characters:', len(text))
# print(text[:1000])

Length of the dataset in characters: 1115394


In [5]:
# Character list:
chars = sorted(list(set(text))) # Sort a list of characters that appear in the text: [, . & .' etc]
vocab_size = len(chars)

print(''.join(chars))
print('Vocabulary size:', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocabulary size: 65


## Tokenize:

In [10]:
# Tokenize: convert raw text (string) to sequence of integers according to some vocabulary of possible elements. 
# Here we translate individual characters (e.g., &) into integers. 

# E.g., Google's using Sentencepiece (not coding whole words nor individual characters - it's sub-word unit level) 
# OpenAI (tiktoken) - example later -. 

# Mapping (look-up table) from characters to integers:
# character : integer and viceversa
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}

# Encode 
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [11]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


#### Example of usage of sub-word encodings (e.g., tiktoken)

In [24]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
# Vocabulary of GPT2
print('Number of tokens in GPT2', enc.n_vocab)

# Encode - list of integers with values between 0 and 50256 
print('Sentence: "hii there"')
print('Encoding:', enc.encode("hii there"))
# Decode:
print('Decoding:', enc.decode(enc.encode("hii there")))

Number of tokens in GPT2 50257
Sentence: "hii there"
Encoding: [71, 4178, 612]
Decoding: hii there


### Encode the entire tiny shakespeare dataset: 

In [27]:
import torch
# Take all the text in tiny shakespeare, encode it and wrap it into a torch.tensor
data = torch.tensor(encode(text), dtype=torch.long)
# Print the shape and dtype of the tensor
print(data.shape, data.dtype)
# Take a look at the first 1k characters
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

### Split data into train and validation sets

In [28]:
# First 90% will be train:
n = int(0.9*len(data)) 
train_data = data[:n]
val_data = data[n:]

In [39]:
block_size = 8 # i.e., context_lenght 
# First 9 (block_size + 1) characters in the training data. 
train_data[:block_size+1] # 8 individual examples
# that is, in the context of 18 -> 47 comes next; in the context of 18,47 -> 56 comes next, etc. 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [44]:
x = train_data[:block_size] # Inputs of the transformer (first block size characters)
# e.g., [18, 47, 56, 57, 58,  1, 15, 47]
y = train_data[1:block_size+1] # Next block size characters -- offset by 1 (y are the targets for each position)
# e.g., [47, 56, 57, 58,  1, 15, 47, 58]

for t in range(block_size):
    context = x[:t+1] # All characters in x up to 't', including 't'. 
    target = y[t]
    print(f"when input if {context} the target: {target}")
    
# We train with context between 1 and all the way up to context of block size. 
# We train it this way not just for computational reasons or efficienty but we want the network 
# to be used to see anything in between as little as one to block size. 
# Useful during inference -- we can start sampling generation as little as 1 character of context.
# After blocksize, we truncate 'cause the transformer will never receive more than block size inputs when it's predicting the next character.

when input if tensor([18]) the target: 47
when input if tensor([18, 47]) the target: 56
when input if tensor([18, 47, 56]) the target: 57
when input if tensor([18, 47, 56, 57]) the target: 58
when input if tensor([18, 47, 56, 57, 58]) the target: 1
when input if tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input if tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input if tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


### Dataloader (train the transformer on chunks of data at a time)

In [55]:
# Now, batches of multiple block_size (done for efficiency, parallel processing of data). 

# We sample random locations of the dataset to pull chunks from  
torch.manual_seed(1337) # To reproducte

batch_size = 4 # Number of independent sequences to be processed in parallel
block_size = 8 # Maximum context length for predictions


def get_batch(split):
    # Generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data # Data array
    # Generate random positions to grab a chunk out of -- generate batch_size number of random offsets. 
    ix = torch.randint(len(data) - block_size, (batch_size,)) # 4 numbers (batch_size = 4), randomly generated between 0 and len(data)-block_size
    x = torch.stack([data[i:i+block_size] for i in ix]) # First block_size characters starting at 'i'
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # Offset by one of x
    # We get these chunks for every one integer 'i' in ix. Stack 1D tensors as rows: 4x8 tensor. 
    return x, y

# Sample a batch (xb, yb):
xb, yb = get_batch('train')
print('Inputs:', xb.shape)
print(xb) # Each row is an independend chunk of the training set
print('Targets:', yb.shape)
print(yb)

print ('-----------')

print('This [4,8] array contain a total of 32 examples, completely independent: \nthat is, in the context of 24 -> 43 comes next; \nin the context of 24, 43 -> 58 comes next, etc.:')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

Inputs: torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets: torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----------
This [4,8] array contain a total of 32 examples, completely independent: 
that is, in the context of 24 -> 43 comes next; 
in the context of 24, 43 -> 58 comes next, etc.:
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [

## Feed

In [56]:
# Input of the transformer: 
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [73]:
# Simplest possible NN (Bigram Language Model):
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337) # For reproducibility

class BigramLanguageModel(nn.Module):
    # Subclass of nn.module 
    # Very simple model, tokens are not talking to each other
    
    def __init__(self, vocab_size):
        super().__init__()
        # Each token directly reads off the logitd for the next token from a look-up table
        # Create a token embedding table of size (vocab size, vocab size): 
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # inputs and targets are passed to the token embedding table
        # e.g., number 24 in xb will plug out the 24th row of the embedding table, number 43 will do the same for 43rd row, etc. 
        # idx and targets are both (B, T) tensor of integers
        # (Batch, Time, Channel) -- (batch_size, block_size, vocab_size) = (4, 8, 64)
        logits = self.token_embedding_table(idx) # (B, T, C) -- Scores for the next character of the sequence -- we're predicting based on the identity of a single token
        
        if targets is None:
            loss = None
        else:
            # Good way to measure the loss or the quality of the predictions is to use the negative log-likelohood (cross-entropy) 
            # Torch expects the channel to be the 2nd dimension -> (B, C, T), so reshape
            B, T, C = logits.shape # (4, 8, 64)
            logits = logits.view(B*T, C) # (4x8, 64)
            # Targets are (B, T):
            targets = targets.view(B*T) # or (-1), size (4x9)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is the current context of some characters in some batch
        # idx is (B, T) array -- e.g., (4, 8)
        
        # Note that this function gets fed with the full idx to predict a single character at a time. 
        # it is only necessary, for predicting the next character, to know the previous one.
        # in line logits[:, -1, :], we're only using the last one instead of the whole history. 
        # so far, no needed. later history will be used. 
        
        # Generate data from the model
        for _ in range(max_new_tokens):
            # Get te predictions for the current idx
            logits, loss = self(idx)
            # Focus only on the last time step
            # logits are (B, T, C) --> convert them to (B, C) 
            # taking the last time-step, as those are the predictions of what comes next.
            logits = logits[:, -1, :] # (B, C)
            # Apply softmax to convert logits to probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the probability distribution, give 1 sample (integer)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1), for each of the batch dimension, we get a single prediction of what comes next. 
            
            # Append sampled index to the running sequence: 
            # whatever is predicted, is concatenated on top of the previous idx along 
            # the first dimension (time dimension) to get (B, T+1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) e.g., -- (4,9) 
        return idx
    
    
# vocab_size = len(chars) characters
m = BigramLanguageModel(vocab_size)
# Passing it the inputs and targets to BLM
# Scores for the next character of the sequence
logits, loss = m(xb, yb)
print(logits.shape)
# We have 65 possible vocab elements, we can guess what the loss should be -- negative log-likelihood : -ln(1/65) = 4.17
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [76]:
idx = torch.zeros((1,1), dtype=torch.long) # (B=1, T=1) dtype int
# (m.generate)[0] to umplug 
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


KuBZvrpxZQgC-hlkq,ptKqHoiX-jjeLJ &slERj KUsBOL!mpJO!zLg'wNfqHAMgq'hZCWhu.W.IBcP 
RFJ&DEs,nw?pxE?xjNH


## Train

In [77]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # takes parameters and updates the gradients

In [85]:
batch_size = 32

for steps in range(10000):
    
    # Get a batch of data from the data loader
    xb, yb = get_batch('train')
    
    # Evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True) # Zeroing the gradients from the previous step 
    loss.backward() # get the gradients from all the parameters
    optimizer.step() # use the gradients to update 
    
print(loss.item())
    

2.443098783493042


In [88]:
idx = torch.zeros((1,1), dtype=torch.long) # (B=1, T=1) dtype int
# (m.generate)[0] to umplug 
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))



INo ompe Pe o m P: somistrou, paburoinsee
Londer wathes 's, mayousorntekeawhart ghruremin whell y angr hinnds oupre hingo, foune:
s hase!
NS:
Stho lereer atiswouese?


He'cindine u'sold ngnfithot pon d te;
Whie roomy t'd ING ELABe ofulin blakl ttr fr s fe
JUn bowid t s r-jure
My avinoursouen te, stht mas t fos.
W:
Inouim s sely loreny goulavir Rie, ak shas askngef e, nghin housuly ot ont armame.
Hel o ppe? mosisigherde w; s tsere cof oubre pl lothes

t re ice'Sat thercorjut nthet, t ey t t:

BE
